In [1]:
import pybullet as p
import math
import time

In [2]:
physicsClientID = p.connect(p.GUI)

p.getConnectionInfo(physicsClientID)

{'isConnected': 1, 'connectionMethod': 1}

In [3]:
import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())

robot = 0
numJoints = 0

In [4]:
def import_urdf():
    # plane = p.loadURDF('plane.urdf')
    global robot, numJoints
    StartPos = [0, 0, 3]
    StartOrientation = p.getQuaternionFromEuler([1.57, 0, 0])
    robot = p.loadURDF("humanoid.urdf", StartPos, StartOrientation, useFixedBase=1)
    position, orientation = p.getBasePositionAndOrientation(robot)
    numBodies = p.getNumBodies()

    numJoints = p.getNumJoints(robot)
    p.setGravity(0, 0, 0)

In [5]:
import_urdf()

In [6]:
for i in range(numJoints):
    if p.getJointInfo(robot, i)[2] == 0:
        print(p.getJointInfo(robot, i)[0], str(
            p.getJointInfo(robot, i)[1]).lstrip('b'))

4 'right_elbow'
7 'left_elbow'
10 'right_knee'
12 'left_hip'
13 'left_knee'
14 'left_ankle'


In [7]:
def joint_control_pos(joint_index, target_angle, bodyID=robot):

    p.setRealTimeSimulation(1)
    p.setJointMotorControl2(bodyIndex=bodyID,
                            jointIndex=joint_index,
                            controlMode=p.POSITION_CONTROL,
                            targetPosition=target_angle)

    # p.setRealTimeSimulation(0)


In [ ]:
def VMAX(jointIndex, timePeriod, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[8]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[9]
    Vmax = -1*((upperLimit - lowerLimit) * math.pi) / timePeriod
    return Vmax

def velocity(timePeriod,Vmax,t):
    frequency = 2*math.pi/timePeriod
    return Vmax*math.sin(frequency*(t))

def joint_control_vel_sin(joint_index, timePeriod = 1, bodyID=robot):
    t=0
    timeStep = 0.01
    maxForce = 1
    vmax = VMAX(joint_index,timePeriod, bodyID)
    p.setRealTimeSimulation(1)
    while(t<=timePeriod):
        targetVel = velocity(timePeriod,vmax,t)
        if(targetVel==0):
            maxForce = 0
        else:
            maxForce = 1*targetVel/abs(targetVel)
        p.setJointMotorControl2(bodyID, joint_index, p.VELOCITY_CONTROL, targetVel, maxForce)
        time.sleep(timeStep)
        t += timeStep


In [ ]:
def ANGLE_limits(jointIndex, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

    return lowerLimit, upperLimit

def targetAngle(lowerLimit, upperLimit, timePeriod, t):
    Angle = (upperLimit - lowerLimit) * abs(math.sin( (2 * math.pi * t) / timePeriod)) + lowerLimit

    return Angle

def joint_control_pos_sin(joint_index, timePeriod = 5, bodyID=robot):

    t=0

    maxForce = 0.01

    lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

    p.setRealTimeSimulation(1)

    while True:
        target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

        p.setJointMotorControl2(bodyID, joint_index, p.POSITION_CONTROL,  target_Angle, maxForce)

        try:
            timeStep = p.getJointState(bodyID, joint_index)[1] / abs((p.getJointState(bodyID, joint_index)[0] - target_Angle))
        except ZeroDivisionError:
            timeStep = 1.41423782742495e-14

        print(timeStep)
        
        time.sleep(timeStep)

        t += timeStep

In [8]:
from datetime import datetime

In [39]:
def ANGLE_limits(jointIndex, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

    return lowerLimit, upperLimit

def targetAngle(lowerLimit, upperLimit, timePeriod, t):
    Angle = (upperLimit - lowerLimit) * abs(math.sin( (2 * math.pi * t) / timePeriod)) + lowerLimit

    return Angle

def joint_control_pos_sin_2(joint_index, timePeriod = 2, bodyID=robot):

    t = 0

    maxForce = 0.02

    lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

    p.setRealTimeSimulation(1)
    i = 0
    while True:

        # time.sleep(0.01)

        target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

        current_angle = p.getJointState(bodyID, joint_index)[0]
        print(target_Angle, current_angle)


        before = datetime.timestamp(datetime.now())

        p.setJointMotorControl2(bodyID, joint_index, p.POSITION_CONTROL,  target_Angle, maxForce)

        while True:

            # time.sleep(1e-03)
            current_angle = float(p.getJointState(bodyID, joint_index)[0])

            if (current_angle >= min(target_Angle-0.001,target_Angle+0.001) and current_angle <= max(target_Angle-0.001,target_Angle+0.001)):
                break
            
        after = datetime.timestamp(datetime.now())

        print(after - before)

        t += 0.25

        i = 2

In [ ]:
p.setRealTimeSimulation(0)

In [ ]:
p.setJointMotorControl2(robot, 9, p.POSITION_CONTROL, 0, 100)

In [25]:
joint_control_vel_sin(10)
p.getJointState(robot,10)

NameError: name 'joint_control_vel_sin' is not defined

In [42]:
joint_control_pos_sin_2(10)

0.0 0.0
0.0009970664978027344
-2.2203152929257595 0.0
0.37599897384643555
-3.14 -2.2193297927676805
0.3419680595397949
-2.2203152929257595 -3.1390038910291866
0.24799799919128418
-3.8453909493226894e-16 -2.2211343499742155
0.27899909019470215
-2.220315292925759 -0.0008854786785031522
0.38000011444091797
-3.14 -2.219345064087423
0.34148311614990234
-2.22031529292576 -3.139003891038856
0.247298002243042
-7.690781898645379e-16 -2.221134349974233
0.28148889541625977
-2.2203152929257586 -0.0008854786785035497
0.38033604621887207
-3.14 -2.2193298416916467
0.3379099369049072
-2.2203152929257617 -3.1390038883450346
0.2467958927154541
-1.1536172847968067e-15 -2.221134349969394
0.2827341556549072
-2.2203152929257604 -0.0008854786785001913
0.3785700798034668
-3.14 -2.2193298534757573
0.3395419120788574
-2.220315292925762 -3.1390038883471205
0.24770808219909668
-1.5381563797290758e-15 -2.221134349969398
0.2823350429534912
-2.2203152929257604 -0.000885478678500578
0.37799787521362305
-3.14 -2.21932

KeyboardInterrupt: 

In [ ]:
p.setJointMotorControl2(bodyID, joint_index, p.POSITION_CONTROL,  target_Angle, maxForce)joint_control_pos(10, -1.57)


In [ ]:
joint_control_pos(10,0)
#reset joint angle

In [40]:
p.resetSimulation()
# go to urdf loading cell

In [41]:
import_urdf()

In [ ]:
lowerLimit = p.getJointInfo(robot, 10)[8]
upperLimit = p.getJointInfo(robot, 10)[9]

In [ ]:
lowerLimit

In [ ]:
upperLimit

In [ ]:
before = datetime.timestamp(datetime.now())

time.sleep(1)

after = datetime.timestamp(datetime.now())

before - after


In [ ]:
p = list(range(1.2,6.9))
print(p)
max(2,4)